In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
SEED = 24

In [38]:
train_raw = pd.read_csv("train.csv")
test_raw = pd.read_csv("test.csv")

# Processing data

## One hot data

In [27]:
df_train = pd.get_dummies(train_raw, columns = ["Sex"])
y = df_train["Age"]
X = df_train.drop(["id","Age"], axis=1)

## Split train & test

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)

## Scalling data

## Test with base linear regression model

In [26]:
base_lr = LinearRegression()
base_lr = base_lr.fit(X_train,y_train)
y_pred = base_lr.predict(X_test)
print("MAE: ", mean_absolute_error(y_test, y_pred))

MAE:  1.4824897525481464


## predict test

In [46]:
def gen_submission(model):
    
    df_sub = test_raw["id"].to_frame()
    
    ## processing
    df_test = pd.get_dummies(test_raw, columns = ["Sex"])
    df_test.drop(["id"], axis=1, inplace=True)
    
    ## gen file 
    pred = base_lr.predict(df_test)
    df_sub["Age"] = pred
    df_sub.to_csv("submission_crab.csv", index =False)

In [47]:
gen_submission(base_lr)